# Train the model.

In [ ]:
%cd maix_train_mx

! python train.py init

In [ ]:
! python train.py -t detector -di ../signal/images -dx ../signal/xml -ep 200 -ap 0.75 -bz 8 train

# Package the trained model into a format compatible with the K210 and place the file in the root directory for download.

In [ ]:
import os
import zipfile
from pathlib import Path

def get_sorted_directories(base_dir):
    # Get all directories in the base directory and sort them by modification time
    dirs = [d for d in os.listdir(base_dir) if (base_dir / d).is_dir()]
    dirs = sorted(dirs, key=lambda d: (base_dir / d).stat().st_mtime, reverse=True)
    return dirs

def main():
    base_dir = Path("/app/maix_train_mx/out")
    
    # Get the sorted list of directories
    sorted_dirs = get_sorted_directories(base_dir)
    
    # Filter out directories that do not contain detector_result.zip
    valid_dirs = [d for d in sorted_dirs if (base_dir / d / "detector_result.zip").exists()]
    
    if not valid_dirs:
        print("No directories contain detector_result.zip")
        return
    
    # Get the most recent directory
    latest_dir = valid_dirs[0]
    latest_dir_path = base_dir / latest_dir
    
    # Unzip detector_result.zip
    zip_path = latest_dir_path / "detector_result.zip"
    print(f"Use '{zip_path}'.")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(latest_dir_path)
    
    # Create a new package file flash-list.kfpkg
    mx_kmodel_path = latest_dir_path / "mx.kmodel"
    flash_list_path = Path("/app/maix_train_mx/flash-list.json")  # Path to the uploaded flash-list.json file
    
    if not (mx_kmodel_path.exists() and flash_list_path.exists()):
        print("mx.kmodel or flash-list.json file does not exist")
        return
    
    # Write mx.kmodel and flash-list.json to flash-list.kfpkg
    output_kfpkg_path = Path("/app") / "flash-list.kfpkg"
    with zipfile.ZipFile(output_kfpkg_path, 'w') as kfpkg:
        kfpkg.write(mx_kmodel_path, "mx.kmodel")
        kfpkg.write(flash_list_path, "flash-list.json")
    
    print(f"flash-list.kfpkg has been generated at {output_kfpkg_path}")

if __name__ == "__main__":
    main()
